<a href="https://colab.research.google.com/github/Ananya-22/CAMS/blob/main/Basic_code%2B1CAMS_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing and installation

In [1]:
# Installations
import sys
if 'google.colab' in sys.modules:
    !pip install emoji --upgrade
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*
    !python -m spacy download en_core_web_lg
    !pip install pyldavis
    !pip install gensim
    !pip install chart_studio
    !pip install --upgrade autopep8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=206a230554c4b8287a98c9871bbd96425a803691369a7c130f0131e122a840e6
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.1/353.1 KB 25.9 MB/s eta 0:00:00
  Attempting uninstall: multimethod
    Found existing installation: multimethod 1.9.1
    Uninstalling multimethod-1.9.1:
      Successfully uninstalled mu

In [2]:
#installing tweet-preprocessor
!pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
from collections import Counter

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

#Natural Language Processing (NLP)
import spacy
import gensim
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.9/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.9/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)


#Dataset upload

In [4]:
df = pd.read_csv('text,category,explanation.txt')
df.columns

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['text', 'category', 'explanation'], dtype='object')

In [ ]:
# df=df.drop(['Unnamed: 0' , 'post_id' , 'post_created','user_id','followers', 'friends', 'favourites', 'statuses', 'retweets'],axis=1)
# df.rename(columns = {'post_text':'original_tweets'}, inplace = True)

In [6]:
df.rename(columns = {'text':'original_tweets'}, inplace = True)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
df.head(20)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,original_tweets,category,explanation
0,"in the end we all die, nothing we do means any...",5.0,"feel more and more empty,fear lonliness,get ti..."
1,Today would have been my best friend's 18th bi...,4.0,"best friend's birthday, he's gone, never supp..."
2,"So I couldn't feel any worse, and I've had eno...",5.0,"absolute minimal contact with, be more direct..."
3,I am 22 and have never had a girlfriend and ha...,4.0,"never had a girlfriend, rejected countless times"
4,I am almost certain that my depression is caus...,4.0,"eating out with family, what after that? Paren..."
5,I live in a country that I don't want to live ...,2.0,"no job, no education or even goals"
6,"EDIT: Wow that many upvotes and support, thank...",3.0,"depression, anxiety, severe OCD, tooth ache, m..."
7,"Hey guys,\r\nI've been depressed, anxious and ...",4.0,she dumped me she laughed and smiled and trea...
8,"Hi guys,\r\nNever even used reddit before but ...",3.0,an unexpected diagnosis after a collapse trig...
9,i always thought hanging yourself would be ext...,0.0,NaN


#Data cleaning

In [8]:
slang_abbrev_dict = {
    'AFAIK': 'As Far As I Know',
    'AFK': 'Away From Keyboard',
    'ASAP': 'As Soon As Possible',
    'ATK': 'At The Keyboard',
    'ATM': 'At The Moment',
    'A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard',
    'BBL': 'Be Back Later',
    'BBS': 'Be Back Soon',
    'BFN': 'Bye For Now',
    'B4N': 'Bye For Now',
    'BRB': 'Be Right Back',
    'BRT': 'Be Right There',
    'BTW': 'By The Way',
    'B4': 'Before',
    'B4N': 'Bye For Now',
    'CU': 'See You',
    'CUL8R': 'See You Later',
    'CYA': 'See You',
    'FAQ': 'Frequently Asked Questions',
    'FC': 'Fingers Crossed',
    'FWIW': 'For What It\'s Worth',
    'FYI': 'For Your Information',
    'GAL': 'Get A Life',
    'GG': 'Good Game',
    'GN': 'Good Night',
    'GMTA': 'Great Minds Think Alike',
    'GR8': 'Great!',
    'G9': 'Genius',
    'IC': 'I See',
    'ICQ': 'I Seek you',
    'ILU': 'I Love You',
    'IMHO': 'In My Humble Opinion',
    'IMO': 'In My Opinion',
    'IOW': 'In Other Words',
    'IRL': 'In Real Life',
    'KISS': 'Keep It Simple, Stupid',
    'LDR': 'Long Distance Relationship',
    'LMAO': 'Laugh My Ass Off',
    'LOL': 'Laughing Out Loud',
    'LTNS': 'Long Time No See',
    'L8R': 'Later',
    'MTE': 'My Thoughts Exactly',
    'M8': 'Mate',
    'NRN': 'No Reply Necessary',
    'OIC': 'Oh I See',
    'OMG': 'Oh My God',
    'PITA': 'Pain In The Ass',
    'PRT': 'Party',
    'PRW': 'Parents Are Watching',
    'QPSA?': 'Que Pasa?',
    'ROFL': 'Rolling On The Floor Laughing',
    'ROFLOL': 'Rolling On The Floor Laughing Out Loud',
    'ROTFLMAO': 'Rolling On The Floor Laughing My Ass Off',
    'SK8': 'Skate',
    'STATS': 'Your sex and age',
    'ASL': 'Age, Sex, Location',
    'THX': 'Thank You',
    'TTFN': 'Ta-Ta For Now!',
    'TTYL': 'Talk To You Later',
    'U': 'You',
    'U2': 'You Too',
    'U4E': 'Yours For Ever',
    'WB': 'Welcome Back',
    'WTF': 'What The Fuck',
    'WTG': 'Way To Go!',
    'WUF': 'Where Are You From?',
    'W8': 'Wait',
    '7K': 'Sick:-D Laugher'
}

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
import preprocessor as p

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# spell = SpellChecker()

# def correct_spellings(text):
#     corrected_text = []
#     misspelled_words = spell.unknown(text.split())
#     for word in text.split():
#         if word in misspelled_words:
#             corrected_text.append(spell.correction(word))
#         else:
#             corrected_text.append(word)
#     return " ".join(corrected_text)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
def remove_emoji(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji_data.unicode_emoji]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

# def remove_emoji(text):
#     emoji_pattern = re.compile(
#         "["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#         u"\U00002702-\U000027B0"
#         u"\U000024C2-\U0001F251"
#         "]+",
#         flags=re.UNICODE)
#     return emoji_pattern.sub(r'', text)


def url_free_text(text):    
    # Cleans text from urls
    text = re.sub(r'http\S+', '', text)
    return text

def username_free_text(text):
    # remove @username from tweets
    text = re.sub('@[\w]+','',text)
    # remove hashtags
    text = re.sub(r'#\w+ ?', '', text)
    # #remove reserved word such as RT,FAV
    # text= p.OPT.RESERVED(text)
    return text

def decontracted(phrase):
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n't", " not", phrase) 
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


def unslang(text):
    # Converts text like "OMG" into "Oh my God"
  
    if text.upper() in slang_abbrev_dict.keys():
        return slang_abbrev_dict[text.upper()]
    else:
        return text

def remove_punc(text):
    # remove numbers
    text_nonum = re.sub(r'\d+', '', text)

    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    
    # substitute multiple whitespace with single whitespace Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace
    

# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: remove_emoji(x)

# Apply `call_emoji_free` which calls the function to remove all emoji's
# df['emoji_free_tweets'] = df['original_tweets'].apply(remove_emoji)
df['original_tweets'] = df['original_tweets'].astype(str)

#Create a new column with url free tweets
df['url_free_tweets'] = df['original_tweets'].apply(url_free_text)

#Create a new column with username free tweets
df['username_free_tweets'] = df['url_free_tweets'].apply(username_free_text)

#Create a new column with removing can't with cannot tweets
df['slang_free_tweets'] = df['username_free_tweets'].apply(decontracted)

#Create a new column  removing OMG with oh my god tweets
df['slang_free_tweets'] = df['slang_free_tweets'].apply(unslang)

#Create a new column with no pun
df['punc_free_tweets'] = df['slang_free_tweets'].apply(remove_punc)




/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:34: DeprecationWarning: invalid escape sequence \w
<>:71: DeprecationWarning: invalid escape sequence \s
<>:34: DeprecationWarning: invalid escape sequence \w
<>:71: DeprecationWarning: invalid escape sequence \s
<ipython-input-11-34ab5a99d751>:34: DeprecationWarning: invalid escape sequence \w
  text = re.sub('@[\w]+','',text)
<ipython-input-11-34ab5a99d751>:71: DeprecationWarning: invalid escape sequence \s
  text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()


In [12]:
df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,original_tweets,category,explanation,url_free_tweets,username_free_tweets,slang_free_tweets,punc_free_tweets
0,"in the end we all die, nothing we do means any...",5.0,"feel more and more empty,fear lonliness,get ti...","in the end we all die, nothing we do means any...","in the end we all die, nothing we do means any...","in the end we all die, nothing we do means any...",in the end we all die nothing we do means anyt...
1,Today would have been my best friend's 18th bi...,4.0,"best friend's birthday, he's gone, never supp...",Today would have been my best friend's 18th bi...,Today would have been my best friend's 18th bi...,Today would have been my best friend is 18th b...,today would have been my best friend is th bir...
2,"So I couldn't feel any worse, and I've had eno...",5.0,"absolute minimal contact with, be more direct...","So I couldn't feel any worse, and I've had eno...","So I couldn't feel any worse, and I've had eno...","So I could not feel any worse, and I have had ...",so i could not feel any worse and i have had e...
3,I am 22 and have never had a girlfriend and ha...,4.0,"never had a girlfriend, rejected countless times",I am 22 and have never had a girlfriend and ha...,I am 22 and have never had a girlfriend and ha...,I am 22 and have never had a girlfriend and ha...,i am and have never had a girlfriend and have ...
4,I am almost certain that my depression is caus...,4.0,"eating out with family, what after that? Paren...",I am almost certain that my depression is caus...,I am almost certain that my depression is caus...,I am almost certain that my depression is caus...,i am almost certain that my depression is caus...


#Data preprocessing

In [13]:
# Load spacy
# Make sure to restart the runtime after running installations and libraries tab
nlp = spacy.load('en_core_web_lg')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)


# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']

# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)


tokens = []

for doc in tokenizer.pipe(df['punc_free_tweets'], batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in STOP_WORDS:
            doc_tokens.append(token.text.lower())   
    tokens.append(doc_tokens)

# Makes tokens column
df['tokens'] = tokens
     

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# Make tokens a string again
df['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df['tokens']]

def get_lemmas(text):
    '''Used to lemmatize the processed tweets'''
    lemmas = []
    
    doc = nlp(text)
    
    # Something goes here :P
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

df['lemmas'] = df['tokens_back_to_text'].apply(get_lemmas)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# Make lemmas a string again
df['lemmas_back_to_text'] = [' '.join(map(str, l)) for l in df['lemmas']]

# Tokenizer function
def tokenize(text):
    """
    Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out
    """
    # Removing url's
    pattern = r"http\S+"
    
    tokens = re.sub(pattern, "", text) # https://www.youtube.com/watch?v=O2onA4r5UaY
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    tokens = re.sub('\w*\d\w*', '', text) # Remove words containing numbers
    # tokens = re.sub('@*!*$*', '', text) # Remove @ ! $
    tokens = tokens.strip(',') # TESTING THIS LINE
    tokens = tokens.strip('?') # TESTING THIS LINE
    tokens = tokens.strip('!') # TESTING THIS LINE
    tokens = tokens.strip("'") # TESTING THIS LINE
    tokens = tokens.strip(".") # TESTING THIS LINE

    tokens = tokens.lower().split() # Make text lowercase and split it
    
    return tokens

# Apply tokenizer
df['lemma_tokens'] = df['lemmas_back_to_text'].apply(tokenize)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:19: DeprecationWarning: invalid escape sequence \w
<>:19: DeprecationWarning: invalid escape sequence \w
<ipython-input-16-0214d658a4f1>:19: DeprecationWarning: invalid escape sequence \w
  tokens = re.sub('\w*\d\w*', '', text) # Remove words containing numbers


In [17]:
df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,original_tweets,category,explanation,url_free_tweets,username_free_tweets,slang_free_tweets,punc_free_tweets,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens
0,"in the end we all die, nothing we do means any...",5.0,"feel more and more empty,fear lonliness,get ti...","in the end we all die, nothing we do means any...","in the end we all die, nothing we do means any...","in the end we all die, nothing we do means any...",in the end we all die nothing we do means anyt...,"[end, die, means, throught, life, endure, bull...",end die means throught life endure bullshit ex...,"[end, die, mean, throught, life, endure, bulls...",end die mean throught life endure bullshit exp...,"[end, die, mean, throught, life, endure, bulls..."
1,Today would have been my best friend's 18th bi...,4.0,"best friend's birthday, he's gone, never supp...",Today would have been my best friend's 18th bi...,Today would have been my best friend's 18th bi...,Today would have been my best friend is 18th b...,today would have been my best friend is th bir...,"[today, best, friend, th, birthday, going, tim...",today best friend th birthday going time sitti...,"[today, good, friend, th, birthday, go, time, ...",today good friend th birthday go time sit make...,"[today, good, friend, th, birthday, go, time, ..."
2,"So I couldn't feel any worse, and I've had eno...",5.0,"absolute minimal contact with, be more direct...","So I couldn't feel any worse, and I've had eno...","So I couldn't feel any worse, and I've had eno...","So I could not feel any worse, and I have had ...",so i could not feel any worse and i have had e...,"[feel, worse, tonight, midnight, going, messag...",feel worse tonight midnight going message girl...,"[feel, bad, tonight, midnight, go, message, gi...",feel bad tonight midnight go message girl abso...,"[feel, bad, tonight, midnight, go, message, gi..."
3,I am 22 and have never had a girlfriend and ha...,4.0,"never had a girlfriend, rejected countless times",I am 22 and have never had a girlfriend and ha...,I am 22 and have never had a girlfriend and ha...,I am 22 and have never had a girlfriend and ha...,i am and have never had a girlfriend and have ...,"[girlfriend, rejected, countless, times, coupl...",girlfriend rejected countless times couples fe...,"[girlfriend, reject, countless, time, couple, ...",girlfriend reject countless time couple feel d...,"[girlfriend, reject, countless, time, couple, ..."
4,I am almost certain that my depression is caus...,4.0,"eating out with family, what after that? Paren...",I am almost certain that my depression is caus...,I am almost certain that my depression is caus...,I am almost certain that my depression is caus...,i am almost certain that my depression is caus...,"[certain, depression, caused, bullshit, shapes...",certain depression caused bullshit shapes size...,"[certain, depression, cause, bullshit, shape, ...",certain depression cause bullshit shape size g...,"[certain, depression, cause, bullshit, shape, ..."


#Word cloud

In [ ]:
from wordcloud import WordCloud
#Frequency of words
fdist = FreqDist(tweets['Segmented#'])
#WordCloud
wc = WordCloud(width=800, height=400, max_words=50).generate_from_frequencies(fdist)
plt.figure(figsize=(12,10))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

#Topic Modelling

In [18]:
# Create a id2word dictionary
id2word = Dictionary(df['lemma_tokens'])
print(len(id2word))
     

# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))
     

# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in df['lemma_tokens']]
     

# Instantiating a Base LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)
     

# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]
     

# Create Topics
topics = [' '.join(t[0:10]) for t in words]
     

# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")
     

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


16582
7703


------ Topic 0 ------
time go know life feel help think friend day talk

------ Topic 1 ------
feel life think time thing know people good day depression

------ Topic 2 ------
feel year go friend life time know day not people

------ Topic 3 ------
feel know i‚äôm don‚äôt think people friend thing life time

------ Topic 4 ------
year feel know life think work time new go people



In [19]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -7.161994797233211

Coherence Score:  0.31758733549214


In [21]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(base_model, corpus, id2word)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.040270  0.011080       1        1  28.691390
2     -0.009813  0.004053       2        1  25.850242
0     -0.030290 -0.006757       3        1  18.680467
1      0.010297 -0.033656       4        1  13.518304
4     -0.010463  0.025280       5        1  13.259597, topic_info=         Term         Freq        Total Category  logprob  loglift
3077    i‚äôm  2026.000000  2026.000000  Default  30.0000  30.0000
104      year  3568.000000  3568.000000  Default  29.0000  29.0000
3355  don‚äôt  1453.000000  1453.000000  Default  28.0000  28.0000
47       life  3762.000000  3762.000000  Default  27.0000  27.0000
98        new  1012.000000  1012.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
33     friend   240.523193  2972.920071   Topic5  -5.1935  -0.4940
655      talk   193.597963  1800.169049   Topic5  -5.4106  -0.2094
48       live   176.188616  1537.315998   Topic5  -5.5048  -0.1458
420       try   182.562128  2392.032607   Topic5  -5.4693  -0.5524
570      well   172.661363  1466.092572   Topic5  -5.5250  -0.1186

[459 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
5731      5  0.778750   accusation
412       1  0.301723     actually
412       2  0.192980     actually
412       3  0.197575     actually
412       4  0.117932     actually
...     ...       ...          ...
5494      5  0.068409           äú
5589      4  0.846452        äúare
7274      1  0.821565  äúnormal‚äù
6993      1  0.880354         äúoh
6515      1  0.771386         üòî

[1097 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2, 5])

In [22]:
vectorizer = CountVectorizer()
data_vectorized = vectorizer.fit_transform(df['lemmas_back_to_text'])
     

# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
             estimator=LatentDirichletAllocation(batch_size=128, 
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1, 
                                                 learning_decay=0.7, 
                                                 learning_method=None,
                                                 learning_offset=10.0, 
                                                 max_doc_update_iter=100, 
                                                 max_iter=10,
                                                 mean_change_tol=0.001, 
                                                 n_components=10, 
                                                 n_jobs=1,
                                                 perp_tol=0.1, 
                                                 random_state=None,
                                                 topic_word_prior=None, 
                                                 total_samples=1000000.0, 
                                                 verbose=0),
             iid=True, n_jobs=1,
             param_grid={'n_topics': [10, 15, 20, 30], 
                         'learning_decay': [0.5, 0.7, 0.9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
             scoring=None, verbose=0)
     

# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))
     

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyboardInterrupt: ignored

In [ ]:
#Defining a function to loop over number of topics to be used to find an 
#optimal number of tipics
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the 
    LDA model with respective number of topics
    """
    coherence_values_topic = []
    model_list_topic = []
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list_topic.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values_topic.append(coherencemodel.get_coherence())

    return model_list_topic, coherence_values_topic    
     

# Can take a long time to run.
model_list_topic, coherence_values_topic = compute_coherence_values(dictionary=id2word,
                                                        corpus=corpus,
                                                        texts=df['lemma_tokens'],
                                                        start=2, limit=200, step=6)
     

In [ ]:
model_5_2 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=68,
                       random_state=42,
                       chunksize=2000,
                       passes=25,
                       decay=0.5,
                       iterations=70)